## _*AI Assignment #3*_

Name|ID
-|-------------
Louai Nasr Zahran|19016198
ِAbdelrahman Ahmed Bahaa|19015882
Gamal Abdelhamed Nasef|19015550
Mohamed Ayman Saeed|19016250




In [ ]:
# Required imports
import math
import time
from tabulate import tabulate
import numpy as np

# The reward values for each cell
# The upper left cell is initialized to a random value and edited afterwards 
# to take each of the values [100, -3, 0, 3] as required
reward = [[0, -1, 10],
        [-1, -1, -1],
        [-1, -1, -1]]

In [ ]:
# Implementation of value iteration using the bellman equations
# r -> value of upper left corner
# theta -> cell difference on which we terminate the algorithm
# discount -> gamma in bellman equations
# returns: optimal policy with obtained values in last iteration
def value_iteration(r: float, theta: float, discount: float):

  # Initialize policy and values lists
  policy = [["", "", ""] for i in range(3)]
  value = [[0, 0, 0] for i in range(3)]

  # Update R
  reward[0][0] = r

  # delta is the max of the actual measured differences between the value of
  # the current iteration and the previous iteration
  delta = math.inf

  # loop until delta becomes smaller than desired theta
  while delta > theta:

    # prepare for new iteration
    previous_value = value
    value = [[0, 0, 0] for i in range(3)]
    delta = 0

    # loop over all states
    for i in range(3):
      for j in range(3):

        # ignore terminal states
        if i == 0 and (j == 0 or j == 2):
          policy[i][j] = "Exit"
          continue

        # loop over all possible actions
        for action in ["L", "R", "U", "D"]:
          val = 0
          all_actions = get_possible_actions(action)

          # loop over all possible outcomes for desired action
          for possible_action in all_actions:
            if possible_action == action:
              prob = 0.8
            else:
              prob = 0.1
            
            next_i, next_j = get_new_state(i, j, possible_action)

            # apply bellman equation
            val += prob * (reward[next_i][next_j] + discount * previous_value[next_i][next_j])

          # update new iteration's delta, values, and policy
          delta = max(delta, val - previous_value[i][j])

          if val > value[i][j]:
            value[i][j] = val
            policy[i][j] = action

  # fix the value of terminal states (as they initialized to 0)
  value[0][0] = r
  value[0][2] = 10
  return value, policy

In [ ]:
def policy_iteration(r: float, theta: float, discount: float):
  # Initialization (same as value iteration, but assign random initial policy)
  randomArr= np.random.random((3,3))
  policy = [["", "", ""] for i in range(3)]
  
  for i in range(3):
    for j in range(3):
      policy[i][j] = getRandomState(randomArr[i][j])

  policy[0][0] = "Exit"
  policy[0][2] = "Exit"

  value = [[0, 0, 0] for i in range(3)]
  reward[0][0] = r
  delta = math.inf
  policy_changed = True

  # loop until the policy doesn't change for two iterations
  while policy_changed:
    policy_changed = False
    
    # Policy Evaluation
    while delta > theta:
      previous_value = value
      delta = 0

      for i in range(3):
        for j in range(3):
          if i == 0 and (j == 0 or j == 2):
            continue

          action = policy[i][j]
          val = 0
          all_actions = get_possible_actions(action)
          for possible_action in all_actions:
            if possible_action == action:
              prob = 0.8
            else:
              prob = 0.1
            
            next_i, next_j = get_new_state(i, j, possible_action)
            val += prob * (reward[next_i][next_j] + discount * previous_value[next_i][next_j])

          delta = max(delta, val - previous_value[i][j])

          value[i][j] = val


    # Policy Improvement
    for i in range(3):
      for j in range(3):
        if i == 0 and (j == 0 or j == 2):
          continue

        for action in ["L", "R", "U", "D"]:
          val = 0
          all_actions = get_possible_actions(action)
          for possible_action in all_actions:
            if possible_action == action:
              prob = 0.8
            else:
              prob = 0.1
            
            next_i, next_j = get_new_state(i, j, possible_action)
            val += prob * (reward[next_i][next_j] + discount * previous_value[next_i][next_j])

          if val > value[i][j]:
            value[i][j] = val
            if action != policy[i][j]:
              policy_changed = True
              policy[i][j] = action

  value[0][0] = r
  value[0][2] = 10


  return value, policy

In [ ]:
def getRandomState(i):
  if i<0.25:
    return "U"
  elif i<0.5:
    return "R"
  elif i<0.75:
    return "D"
  else:
    return "L"

In [ ]:
def get_new_state(i: int, j: int, action: str):
  if action == "U":
    new_i = i - 1
    new_j = j
  if action == "D":
    new_i = i + 1
    new_j = j
  if action == "L":
    new_i = i
    new_j = j - 1
  if action == "R":
    new_i = i
    new_j = j + 1
  
  if new_i == -1 or new_i == 3 or new_j == -1 or new_j == 3:
    return i, j
  return new_i, new_j

In [ ]:
def get_possible_actions(action: str):
  if action == "U":
    return ["U", "R", "L"]
  if action == "L":
    return ["L", "U", "D"]
  if action == "R":
    return ["R", "U", "D"]
  if action == "D":
    return ["D", "L", "R"]
  
  return []

In [ ]:
t = time.time()
rArr=[100,3,0,-3]
for r in rArr:
  print("===================================================")
  print("r = "+str(r))
  value, policy = value_iteration(r, 0.01, 0.99)
  print("Time taken value: " + str(time.time() - t))
  print(tabulate(value))
  print(tabulate(policy))
  print()

  t = time.time()
  value, policy = policy_iteration(r, 0.01, 0.99)
  print("Time taken policy: " + str(time.time() - t))
  print(tabulate(value))
  print(tabulate(policy))
  print()

r = 100
Time taken value: 0.0057370662689208984
--------  -------  -------
100       99.1954  10
 99.1954  96.7181  90.0975
 96.4459  94.295   91.6752
--------  -------  -------
----  -  ----
Exit  L  Exit
U     L  D
U     L  L
----  -  ----

Time taken policy: 0.0025599002838134766
--------  -------  -------
100       99.0945  10
 99.0939  96.51    88.1159
 96.1524  93.9246  90.9992
--------  -------  -------
----  -  ----
Exit  L  Exit
U     L  D
U     L  L
----  -  ----

r = 3
Time taken value: 0.011793851852416992
-------  -------  --------
3        9.55728  10
6.44537  8.19421   9.55728
5.62534  6.86024   8.04434
-------  -------  --------
----  -  ----
Exit  R  Exit
R     R  U
R     R  U
----  -  ----

Time taken policy: 0.0008528232574462891
-------  -------  --------
3        9.54739  10
6.35029  8.17335   9.55431
5.47879  6.82343   8.03595
-------  -------  --------
----  -  ----
Exit  R  Exit
R     R  U
R     R  U
----  -  ----

r = 0
Time taken value: 0.0067713260650634766
-

**At r = 100**

*   The agent will likely go towards r as the reward of other states is just -1 and it can take 100 when reaching r.
*   States around the final state that have a reward of *10 points*  will move to the opposite direction to avoid any possibility of going into it.


**At r = 3, 0, -3**

*   The agent will likely go towards final state that have 10 reward as the reward of other cells is just -1 and it can reach the state that has a 10 points reward from any state using a maximum of 4 moves.
*   The arrow in the state below r is pointing to right going to state 10 as fast as possible. avoiding to make a detour if it go down as it will cost more than going right.
*   Note for r = 3 we can calculate if we go to r directly from the farthest point from 10 so we can see why it prefers 10 over r=3 for 10 we gain 9.606 (10 * 0.99^4) and for 3 we gain 2.904 (10 * 0.99^2)

### **Note:**
It is worth noting that for each run, the policy iteration produces the same optimal policy as the value iteration, with the policy iteration algorithm being an order of magnitude faster (10x).

